In [1]:
import pandas as pd
import numpy as np
import redefine

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score

In [2]:
from pprint import pprint

In [3]:
data = pd.read_csv('data/iris_modified.csv')

In [4]:
data

,id,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,virginica
146,146,6.3,2.5,5.0,1.9,versicolor
147,147,6.5,3.0,5.2,2.0,virginica
148,148,6.2,3.4,5.4,2.3,virginica


In [5]:
rd = redefine.REDEFINE('iris_modified.csv', data, 'target', 'id')

In [7]:
results = rd.run_redefine('Random Forest', {'n_estimators':'5'}, 'KMeans', {}, 'Standard')

In [31]:
results.to_csv('testing.txt', sep='\t')

In [20]:
X = rd.get_X()
IDs = rd.get_IDs()
Y = rd.get_Y()
Y_names = rd.get_Y_names()
kfolds = 10

In [294]:
def accuracy_score(ytest, yhat):
        nz = np.flatnonzero(ytest == yhat)
        return len(nz)/len(ytest)

def doKFold(model, scaler):
        n = len(X)
        idxs = np.linspace(0, n, kfolds+1).astype(int)
        idx = np.arange(0, n)
        
        np.random.shuffle(idx)

        x = X.copy()[idx]
        y = Y.copy()[idx]
        ids = IDs.copy()[idx]

        accuracy_scores = np.zeros(kfolds)
        
        ####
        idtests = np.zeros(n, dtype='O')
        yhattests = np.zeros(n, dtype='O')
        ####

        for k, i in enumerate(range(1, len(idxs))):
            idx1 = idxs[i-1]
            idx2 = idxs[i]

            xtest = x[idx1:idx2]
            xtrain = np.concatenate([x[idxs[0]:idx1], x[idx2:idxs[-1]]])

            ytest = y[idx1:idx2]
            ytrain = np.concatenate([y[idxs[0]:idx1], y[idx2:idxs[-1]]])

            if scaler is not None:
                xtrain = scaler.fit_transform(xtrain)
                xtest = scaler.transform(xtest)

            model.fit(xtrain, ytrain)
            yhat = model.predict(xtest)
            
            accuracy_scores[k] = accuracy_score(ytest, yhat)
            
            ####
            idtests[idx1:idx2] = ids[idx1:idx2]
            yhattests[idx1:idx2] = yhat
            ####
        #, ####
        return np.mean(accuracy_scores), zip(idtests, yhattests)

In [293]:
testarr = np.zeros(10, dtype='O')
testarr[0:3] = np.array(['1', '1', '1'])
testarr

array(['1', '1', '1', 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [295]:
model = RandomForestClassifier()
scaler = StandardScaler()
score, results = doKFold(model, scaler)

In [296]:
results

In [271]:
score

0.9572463768115942

In [241]:
type(IDs)

pandas.core.series.Series

In [297]:
df = pd.DataFrame(columns=['supervised'], index=IDs)

In [298]:
df

,supervised
ID,
4069430,NaN
4211823,NaN
4215950,NaN
4221545,NaN
4265842,NaN
...,...
5472355,NaN
5472399,NaN
5472565,NaN


In [299]:
for idx, res in results:
    print(idx, res)
    df.at[idx, 'supervised'] = res

12251295 exaltata
4432127 exaltata
23241200 exaltata
4211846 syriaca
5470355 exaltata
23241212 exaltata
4432416 syriaca
23241172 exaltata
19605479 exaltata
4432412 syriaca
7137240 exaltata
12251331 exaltata
23241173 exaltata
23241263 exaltata
23241168 exaltata
23141186 exaltata
19941670 exaltata
12327864 exaltata
19605451 exaltata
4221545 exaltata
4432447 syriaca
4432414 syriaca
3285876 syriaca
24721449 exaltata
19605768 exaltata
4069039 syriaca
19330844 exaltata
23241182 exaltata
23241231 exaltata
5069317 syriaca
19941672 exaltata
17021602 exaltata
19605472 exaltata
13297772 exaltata
21623019 exaltata
12251547 exaltata
12251291 exaltata
23241185 exaltata
3285874 syriaca
19605498 exaltata
12251305 exaltata
24721753 exaltata
23241289 exaltata
4600623 syriaca
16891267 exaltata
4432432 syriaca
12318721 exaltata
21166347 exaltata
11164025 exaltata
12251555 exaltata
23241195 exaltata
4217515 syriaca
24721758 exaltata
4265210 syriaca
17171818 exaltata
21623020 exaltata
23241181 exaltata
1329

In [301]:
df

,supervised
ID,
4069430,syriaca
4211823,exaltata
4215950,exaltata
4221545,exaltata
4265842,exaltata
...,...
5472355,syriaca
5472399,exaltata
5472565,exaltata


TypeError: 'NoneType' object is not iterable